In [1]:
#Mounting the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
#importing the necessary libraries.
import re
import nltk
nltk.download('punkt')
re.compile('<title>(.*)</title>')
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
#Initializing lemmatizer object
preProcessLemmatizer = WordNetLemmatizer() 
from nltk import PorterStemmer
from nltk.corpus import stopwords 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
#import the os package to change the directory.
import os
os.chdir("/content/drive/MyDrive/IR_Dataset/Assignment_1/CSE508_Winter2023_Dataset")

In [13]:
#import the glob library to read all the files.
import glob
html_files = glob.glob('*')
print(html_files)

['cranfield0131', 'cranfield0126', 'cranfield0129', 'cranfield0125', 'cranfield0128', 'cranfield0130', 'cranfield0123', 'cranfield0127', 'cranfield0122', 'cranfield0124', 'cranfield0121', 'cranfield0116', 'cranfield0120', 'cranfield0115', 'cranfield0119', 'cranfield0117', 'cranfield0118', 'cranfield0113', 'cranfield0112', 'cranfield0114', 'cranfield0111', 'cranfield0102', 'cranfield0108', 'cranfield0109', 'cranfield0104', 'cranfield0107', 'cranfield0110', 'cranfield0106', 'cranfield0105', 'cranfield0103', 'cranfield0101', 'cranfield0099', 'cranfield0093', 'cranfield0096', 'cranfield0100', 'cranfield0097', 'cranfield0095', 'cranfield0098', 'cranfield0094', 'cranfield0092', 'cranfield0091', 'cranfield0086', 'cranfield0090', 'cranfield0083', 'cranfield0088', 'cranfield0082', 'cranfield0089', 'cranfield0085', 'cranfield0084', 'cranfield0087', 'cranfield0081', 'cranfield0072', 'cranfield0078', 'cranfield0079', 'cranfield0077', 'cranfield0080', 'cranfield0074', 'cranfield0076', 'cranfield007

In [14]:
#printing the length of the files.
len(html_files)

1400

In [ ]:
#Priting first 5 files before performing any operation
first5 = ['cranfield0002', 'cranfield0003', 'cranfield0004', 'cranfield0005', 'cranfield0001']

for filename in first5:
    with open(filename, 'r') as file:
        contents = file.read()
        print(contents)

simple shear flow past flat plate incompressible fluid small viscosity study highspeed viscous flow past twodimensional body usually necessary consider curved shock wave emitting nose leading edge body consequently exists inviscid rotational flow region shock wave boundary layer situation arises instance study hypersonic viscous flow past flat plate situation somewhat different prandtls classical boundarylayer problem prandtls original problem inviscid free stream outside boundary layer irrotational hypersonic boundarylayer problem inviscid free stream must considered rotational possible effects vorticity recently discussed ferri libby present paper simple shear flow past flat plate fluid small viscosity investigated shown problem treated boundarylayer approximation novel feature free stream constant vorticity discussion restricted twodimensional incompressible steady flow
boundary layer simple shear flow past flat plate boundarylayer equations presented steady incompressible flow pres

**(i) Bigram inverted index**

In [15]:
#creating the bigram inverted index
def create_bigram_index(html_files):
    index = {}
    doc_id = 0
    for file_name in html_files:
        with open(file_name, "r") as file:
            doc_id += 1
            document = file.read()
            words = document.split()
            for i in range(len(words) - 1):
                bigram = f"{words[i]} {words[i + 1]}"
                if bigram in index:
                    index[bigram].append(doc_id)
                else:
                    index[bigram] = [doc_id]
    return index

In [17]:
#print the Bigram index
bigram_index = create_bigram_index(html_files)
print(bigram_index)

{'twodimensional jet': [1, 1, 1184], 'jet mixing': [1, 1, 1, 1, 2, 323, 323, 323, 323, 323, 323, 950], 'mixing compressible': [1, 483, 950], 'compressible fluid': [1, 1, 17, 60, 60, 158, 162, 267, 267, 323, 331, 385, 385, 414, 452, 483, 559, 676, 725, 760, 762, 762, 764, 764, 878, 878, 1011, 1251, 1385], 'fluid mixing': [1], 'mixing divergence': [1], 'divergence supersonic': [1, 291, 658], 'supersonic jet': [1, 221, 235, 235, 535, 736, 1016, 1050, 1313, 1313], 'jet exhausting': [1, 217, 616, 734, 739, 935, 935, 938, 938, 946, 946, 1018, 1018, 1018, 1020, 1313, 1313, 1327], 'exhausting supersonic': [1, 1, 1019], 'supersonic stream': [1, 142, 143, 217, 626, 896, 898, 945, 1017, 1017, 1050, 1050, 1187, 1210, 1249, 1287], 'stream investigated': [1, 961], 'investigated theoretically': [1, 500, 810, 899, 1187], 'theoretically first': [1], 'first part': [1, 1, 184, 191, 212, 214, 216, 238, 332, 840, 1266], 'part paper': [1, 1, 184, 834, 840, 840, 1041, 1266], 'paper flow': [1], 'flow assumed'

In [18]:
#save the pickle file
import pickle

def save_index(index, filename):
    with open(filename, 'wb') as f:
        pickle.dump(index, f)

# index = create_bigram_index(html_files)
save_index(bigram_index, 'bigram_index.pkl')

In [19]:
#load the pickle file
def load_index(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

index = load_index('bigram_index.pkl')

**(ii) Positional index**

In [20]:
#creating the positonal index
def create_positional_index(html_files):
    index = {}
    for doc_id, file_name in enumerate(html_files):
      with open(file_name, 'r') as f:
        documents = f.readlines()
        # print(documents)
      # Loop through each document
      for doc in documents:
        # Split the document into words
        words = doc.split()
        # print(words)
        # Loop through each word
        for i, word in enumerate(words):
            # Add the word and its position to the index
            if word not in index:
                index[word] = []
            # print(i)
            # print(word)
            # print(doc_id)
            index[word].append((doc_id, i))

    return index

In [21]:
#print the positonal index
positional_index = create_positional_index(html_files)
print(positional_index)

{'twodimensional': [(0, 0), (0, 36), (0, 54), (0, 79), (0, 158), (1, 1), (1, 36), (4, 11), (16, 24), (38, 35), (38, 55), (38, 82), (41, 29), (46, 23), (59, 10), (72, 17), (74, 67), (77, 46), (85, 25), (100, 92), (101, 10), (102, 2), (102, 21), (102, 69), (103, 15), (103, 106), (111, 58), (115, 7), (116, 0), (125, 125), (133, 12), (142, 6), (156, 13), (159, 62), (164, 20), (172, 24), (172, 48), (172, 57), (176, 13), (182, 39), (182, 178), (187, 135), (188, 47), (189, 8), (189, 23), (191, 16), (192, 16), (193, 9), (207, 84), (211, 16), (211, 21), (211, 101), (213, 16), (213, 20), (213, 117), (215, 82), (216, 171), (236, 13), (237, 27), (257, 21), (271, 26), (274, 13), (276, 12), (280, 21), (280, 130), (281, 35), (288, 32), (290, 29), (297, 6), (297, 51), (357, 110), (359, 19), (360, 12), (390, 68), (391, 12), (401, 48), (404, 66), (406, 1), (412, 42), (415, 72), (419, 19), (419, 31), (419, 40), (429, 11), (429, 45), (431, 62), (441, 18), (453, 42), (455, 19), (458, 11), (468, 11), (473, 

In [22]:
#save the pickle file
def save_index_positional_index(index, filename):
    with open(filename, 'wb') as f:
        pickle.dump(index, f)

# index = create_bigram_index(html_files)
save_index_positional_index(positional_index, 'positional_index.pkl')

In [23]:
#load the pickle file
def load_positional_index(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

index = load_positional_index('positional_index.pkl')

**(iii) Compare and comment on your results using (i) and (ii)**

In [29]:
#preprocess the given query 
def preprocess_query(query):
    # Convert to lowercase
    query = query.lower()

    # Tokenize the query
    query_tokens = nltk.word_tokenize(query)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    query_tokens = [token for token in query_tokens if token not in stop_words]

    # Remove punctuation
    query_tokens = [token for token in query_tokens if re.search('[a-zA-Z]', token)]

    # Remove blank space tokens
    query_tokens = [token for token in query_tokens if len(token) > 0]

    return query_tokens

In [49]:
#function to compare the bigram and positional index
def search_phrase(sentence):
  # extract bigrams from sentence
  bigrams = []
  words = sentence.split()
  for i in range(len(words) - 1):
    bigram = words[i] + " " + words[i+1]
    bigrams.append(bigram)

  # search for bigrams in bigram inverted index
  documents_bigram = []
  for bigram in bigrams:
    if bigram in bigram_index:
        documents_bigram.extend(bigram_index[bigram])

  documents_bigram = list(set(documents_bigram))
  num_docs_bigram = len(documents_bigram) 
  print("Number of documents retrieved for query '{}' using bigram inverted index: {}".format(sentence, num_docs_bigram))
  files_bigram = [html_files[i] for i in documents_bigram]
  print("Names of documents retrieved for query '{}' using bigram inverted index: {}".format(sentence, files_bigram))

  # search for words in positional inverted index
  documents_pos = []
  for word in words:
    if word in positional_index:
        documents_pos.extend([doc_id for doc_id, pos in positional_index[word]])

  documents_pos = list(set(documents_pos))  
  num_docs_pos = len(documents_pos)
  print("Number of documents retrieved for query '{}' using positional inverted index: {}".format(sentence, num_docs_pos))
  files_pos = [html_files[i] for i in documents_pos]
  print("Names of documents retrieved for query '{}' using positional inverted index: {}".format(sentence, files_pos))


In [50]:
#Taking the user input
n = int(input("Enter the number of queries: "))
queries = []
pre_query=''
for i in range(n):
  queries.append(input("Enter query: "))

x=0
for query in queries:
  pre_query=preprocess_query(query)
  final_query=' '.join(pre_query)

  print(final_query)
  search_phrase(final_query)

Enter the number of queries: 1
Enter query: flow of the theory
flow theory
Number of documents retrieved for query 'flow theory' using bigram inverted index: 29
Names of documents retrieved for query 'flow theory' using bigram inverted index: ['cranfield0538', 'cranfield0136', 'cranfield1137', 'cranfield0369', 'cranfield0686', 'cranfield0748', 'cranfield1278', 'cranfield0689', 'cranfield0518', 'cranfield1211', 'cranfield0297', 'cranfield1267', 'cranfield1360', 'cranfield0090', 'cranfield1257', 'cranfield1330', 'cranfield1103', 'cranfield1313', 'cranfield0568', 'cranfield0879', 'cranfield1171', 'cranfield0318', 'cranfield0039', 'cranfield1017', 'cranfield0640', 'cranfield0464', 'cranfield0466', 'cranfield1301', 'cranfield0029']
Number of documents retrieved for query 'flow theory' using positional inverted index: 890
Names of documents retrieved for query 'flow theory' using positional inverted index: ['cranfield0131', 'cranfield0129', 'cranfield0125', 'cranfield0128', 'cranfield0123', 

In [ ]:
flo